In [1]:
import txjupyter
reactor = txjupyter.start('tcp:port=8080')

In [2]:
from klein import route
@route('/')
def home(request):
    return 'Hello, world!'

In [3]:
@route('/foo')
def froop(request):
    return 'Goodbye!'

In [19]:
from twisted.internet import defer
from twisted.python import log
import treq
@defer.inlineCallbacks
def grab_content():
    response = yield treq.get('http://localhost:8080')
    content = yield response.content()
    print(content)

In [20]:
d = grab_content()

Hello, world!
